<a href="https://colab.research.google.com/github/petervinhchau/public/blob/main/ece219_prj4_q10_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
# Super Bowl Fan Classification Project - Fixed Google Colab Version

This notebook implements a data mining project to classify tweets as supporting either the Patriots or Seahawks
during Super Bowl 49, and analyzes how fan language and engagement patterns changed throughout the game.

## Setup and Data Loading
"""

# Install required packages
!pip install pandas matplotlib pytz scikit-learn nltk imblearn seaborn tqdm

import os

# Check if dataset already exists
if not os.path.exists('/content/tweets_dataset.zip'):
    # Download the dataset (tweets data) and unzip it
    !wget -O tweets_dataset.zip -L "https://ucla.box.com/shared/static/24oxnhsoj6kpxhl6gyvuck25i3s4426d"
    !mkdir -p /content/tweet_data/
    !unzip -o tweets_dataset.zip -d /content/tweet_data/
else:
    print("Dataset already exists, skipping download and unzip.")

"""
## Import Libraries
"""

import json
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pytz
import glob
from tqdm.notebook import tqdm

# For text processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
import nltk

# For modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

# For dealing with imbalanced data
from imblearn.over_sampling import RandomOverSampler

# Download required NLTK data
nltk.download('stopwords', quiet=True)
# stop_words = set(stopwords.words('english'))
stop_words = list(stopwords.words('english'))

# Set random seed for reproducibility
np.random.seed(42)


In [ ]:
"""
## Step 1: Find and Load the Dataset
"""

# First, let's find all files in the dataset
print("=== Checking available files ===")
!find /content/tweet_data -type f | sort

# Function to safely load tweets from JSON file
def load_tweets(file_path, max_tweets=None):
    """
    Load tweets from a JSON file where each line is a separate JSON object.

    Parameters:
    - file_path: Path to the JSON file
    - max_tweets: Optional limit on number of tweets to load (for testing)

    Returns:
    - DataFrame containing the parsed tweets
    """
    print(f"Loading tweets from {file_path}...")
    tweets = []
    count = 0
    errors = 0

    with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
        for line in tqdm(f):
            if max_tweets and count >= max_tweets:
                break

            # Skip empty lines
            if not line.strip():
                continue

            try:
                tweet = json.loads(line.strip())
                tweets.append(tweet)
                count += 1
            except json.JSONDecodeError:
                errors += 1
                if errors <= 5:  # Only show first few errors
                    print(f"Error parsing JSON on line {count+errors}")

    print(f"Successfully loaded {len(tweets)} tweets with {errors} errors")
    return pd.DataFrame(tweets)

# Find tweet files and load the first one found
tweet_files = glob.glob('/content/tweet_data/**/tweets_*.txt', recursive=True)

if not tweet_files:
    print("No tweet files found! Please check the dataset extraction.")
    # Create dummy data for testing
    df = pd.DataFrame({
        'text': ['#gopatriots great game', '#gohawks we can win', 'watching #superbowl'],
        'author': [{'followers': 100}, {'followers': 200}, {'followers': 150}],
        'citation_date': [1422832800, 1422836400, 1422840000],  # Feb 1, 2015 timestamps
        'metrics': [{'citations': {'total': 5}}, {'citations': {'total': 10}}, {'citations': {'total': 2}}]
    })
    print("Created dummy data for testing since no files were found.")
else:
    print(f"Found {len(tweet_files)} tweet files")
    # Load the first file
    df = load_tweets(tweet_files[0], max_tweets=50000)  # Adjust max_tweets as needed

# Basic dataset info
print("\n=== Dataset Information ===")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

# Check for basic required columns
required_cols = ['text', 'author', 'citation_date', 'metrics']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"Warning: Missing required columns: {missing_cols}")

    # Try to repair missing columns with dummy values
    for col in missing_cols:
        if col == 'text':
            df['text'] = 'No text available'
        elif col == 'author':
            df['author'] = [{'followers': 0}] * len(df)
        elif col == 'citation_date':
            # Feb 1, 2015 - Super Bowl day
            df['citation_date'] = 1422835200
        elif col == 'metrics':
            df['metrics'] = [{'citations': {'total': 0}}] * len(df)

    print("Added dummy values for missing columns to allow processing to continue")


In [ ]:
# STEP 2

import json
import pandas as pd
import numpy as np
import re
import pytz
from datetime import datetime

###########################################
# Fix tweet text extraction after Step 1
###########################################

# If the loaded DataFrame (df) has a 'tweet' column (with nested tweet data),
# extract the tweet text from it. Otherwise, assume df already has a 'text' column.
if 'tweet' in df.columns:
    # Create a new column 'text' from df['tweet']['text']
    def extract_text(nested):
        if isinstance(nested, dict):
            return nested.get('text', '')
        return ''
    df['text'] = df['tweet'].apply(extract_text)
    print("Extracted tweet text from the nested 'tweet' field.")
else:
    print("Using existing 'text' column in df.")

# Quick debug print: show first 10 tweet texts
print("\n=== Debug: Raw 'text' samples ===")
print(df['text'].head(10))

###########################################
# Preprocessing Functions (Step 2)
###########################################

def clean_text(text):
    """
    Clean tweet text by removing URLs, mentions, hashtags, and special characters.
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+', '', text)        # Remove URLs
    text = re.sub(r'@\S+', '', text)             # Remove mentions
    text = re.sub(r'#\S+', '', text)             # Remove hashtags (or use text.replace('#','') to keep the word)
    text = re.sub(r'[^\w\s]', ' ', text)         # Remove special characters
    text = re.sub(r'\d+', '', text)              # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()     # Remove extra whitespace
    return text

def extract_hashtags(text):
    """
    Extract hashtags from tweet text.
    """
    if not isinstance(text, str):
        return []
    hashtags = re.findall(r'#(\w+)', text.lower())
    return hashtags

def preprocess_tweets(df):
    """
    Preprocess the tweet dataset by:
      1. Converting UNIX timestamps to datetime
      2. Identifying team affiliation based on hashtags and text content
      3. Cleaning tweet text
      4. Extracting additional metadata
    """
    print("\n--- Preprocessing Tweets ---")
    processed_df = df.copy()

    # Extract hashtags (before cleaning text)
    print("Extracting hashtags...")
    processed_df['hashtags'] = processed_df['text'].apply(extract_hashtags)

    # Clean tweet text
    print("Cleaning tweet text...")
    processed_df['clean_text'] = processed_df['text'].apply(clean_text)

    # Convert UNIX timestamp to PST datetime (using 'citation_date')
    print("Converting timestamps...")
    pst_tz = pytz.timezone('America/Los_Angeles')
    def convert_timestamp(ts):
        if pd.isnull(ts):
            return None
        try:
            return datetime.fromtimestamp(float(ts), pst_tz)
        except (ValueError, TypeError, OverflowError):
            return None
    processed_df['datetime'] = processed_df['citation_date'].apply(convert_timestamp)

    # Remove rows with invalid dates
    valid_date_mask = ~processed_df['datetime'].isna()
    if valid_date_mask.sum() < len(processed_df):
        print(f"Warning: {len(processed_df) - valid_date_mask.sum()} tweets have invalid dates and will be removed.")
        processed_df = processed_df[valid_date_mask].copy()

    # Create time features
    processed_df['hour'] = processed_df['datetime'].dt.hour
    processed_df['minute'] = processed_df['datetime'].dt.minute

    # Define game time (example for Super Bowl 49)
    game_start = datetime(2015, 2, 1, 15, 30, tzinfo=pst_tz)
    game_end   = datetime(2015, 2, 1, 19, 0, tzinfo=pst_tz)

    processed_df['time_period'] = 'unknown'
    processed_df.loc[processed_df['datetime'] < game_start, 'time_period'] = 'pre_game'
    processed_df.loc[(processed_df['datetime'] >= game_start) & (processed_df['datetime'] <= game_end), 'time_period'] = 'during_game'
    processed_df.loc[processed_df['datetime'] > game_end, 'time_period'] = 'post_game'

    # Approximate game quarters
    q1_end = datetime(2015, 2, 1, 16, 15, tzinfo=pst_tz)
    q2_end = datetime(2015, 2, 1, 17, 0, tzinfo=pst_tz)
    q3_end = datetime(2015, 2, 1, 17, 45, tzinfo=pst_tz)
    processed_df['quarter'] = 'not_during_game'
    processed_df.loc[(processed_df['datetime'] >= game_start) & (processed_df['datetime'] < q1_end), 'quarter'] = 'q1'
    processed_df.loc[(processed_df['datetime'] >= q1_end) & (processed_df['datetime'] < q2_end), 'quarter'] = 'q2'
    processed_df.loc[(processed_df['datetime'] >= q2_end) & (processed_df['datetime'] < q3_end), 'quarter'] = 'q3'
    processed_df.loc[(processed_df['datetime'] >= q3_end) & (processed_df['datetime'] <= game_end), 'quarter'] = 'q4'

    # Identify team affiliations based on hashtags and text content
    print("Identifying team affiliations...")
    processed_df['team'] = 'neutral'

    # Define patterns (adjust these as needed)
    patriots_patterns = ['gopatriots', 'patriots', 'pats', 'tombrady', 'gopats', 'patriotsnation', 'patsnation']
    seahawks_patterns = ['gohawks', 'seahawks', 'seattleseahawks', 'goseahawks', '12s', '12thman', 'hawksnation']

    # Create masks based on text content
    patriots_mask = processed_df['text'].str.lower().str.contains('|'.join(patriots_patterns), regex=True)
    seahawks_mask = processed_df['text'].str.lower().str.contains('|'.join(seahawks_patterns), regex=True)
    print(f"Number of tweets matching patriots text pattern: {patriots_mask.sum()}")
    print(f"Number of tweets matching seahawks text pattern: {seahawks_mask.sum()}")

    # Create masks based on hashtags
    hashtags_pats = processed_df['hashtags'].apply(lambda tags: any(pat in tags for pat in patriots_patterns))
    hashtags_hawks = processed_df['hashtags'].apply(lambda tags: any(pat in tags for pat in seahawks_patterns))
    print(f"Number of tweets matching patriots hashtag: {hashtags_pats.sum()}")
    print(f"Number of tweets matching seahawks hashtag: {hashtags_hawks.sum()}")

    # Assign teams
    processed_df.loc[patriots_mask | hashtags_pats, 'team'] = 'patriots'
    seahawks_unassigned = (processed_df['team'] == 'neutral') & (seahawks_mask | hashtags_hawks)
    processed_df.loc[seahawks_unassigned, 'team'] = 'seahawks'

    # Extract metadata
    print("Extracting tweet metadata...")
    def extract_followers(author_obj):
        if not isinstance(author_obj, dict):
            return 0
        return author_obj.get('followers', 0) or 0
    def extract_retweets(metrics_obj):
        if not isinstance(metrics_obj, dict):
            return 0
        citations = metrics_obj.get('citations', {})
        if not isinstance(citations, dict):
            return 0
        return citations.get('total', 0) or 0
    processed_df['follower_count'] = processed_df['author'].apply(extract_followers)
    processed_df['retweet_count'] = processed_df['metrics'].apply(extract_retweets)

    # Filter for team-affiliated tweets only
    team_tweets = processed_df[processed_df['team'] != 'neutral'].copy()
    print(f"Found {len(team_tweets)} tweets with team affiliations")
    print("\nTeam distribution:")
    print(team_tweets['team'].value_counts())

    if len(team_tweets) < 100:
        print("\nWARNING: Very few team-affiliated tweets found. This might affect model quality.")
        top_hs = processed_df['hashtags'].explode().value_counts().head(20)
        print("\nTop hashtags after cleaning/extraction:")
        print(top_hs)

    return team_tweets

###########################################
# Run Preprocessing
###########################################
team_tweets = preprocess_tweets(df)

print("\n--- Final Preprocessed Data Statistics ---")
print(f"Number of Patriots tweets: {(team_tweets['team'] == 'patriots').sum()}")
print(f"Number of Seahawks tweets: {(team_tweets['team'] == 'seahawks').sum()}")

print("\nTweets by time period:")
print(team_tweets['time_period'].value_counts())

print("\nTweets by quarter:")
print(team_tweets['quarter'].value_counts())

print("\nSample of cleaned tweets:")
for team in ['patriots', 'seahawks']:
    subset = team_tweets[team_tweets['team'] == team]
    if len(subset) > 0:
        sample_size = min(3, len(subset))
        tweets = subset['clean_text'].sample(sample_size).tolist()
        print(f"\n{team.upper()} fan tweets:")
        for t in tweets:
            print(f"- {t}")


In [ ]:
"""
## Step 3: Feature Engineering
"""

print("team_tweets shape:", team_tweets.shape)
print("Number of Patriots tweets:", (team_tweets['team'] == 'patriots').sum())
print("Number of Seahawks tweets:", (team_tweets['team'] == 'seahawks').sum())
print("Number of empty clean_text:", (team_tweets['clean_text'].str.strip() == '').sum())


def engineer_features(team_tweets):
    """
    Engineer features for the classification task:
    1. TF-IDF features from cleaned text
    2. Additional metadata features

    Parameters:
    - team_tweets: DataFrame containing preprocessed tweets

    Returns:
    - X: Feature matrix
    - y: Target labels
    - feature_names: Names of the features
    """
    print("\n--- Feature Engineering ---")

    # Ensure we have required columns
    required_cols = ['clean_text', 'team', 'time_period', 'quarter', 'follower_count', 'retweet_count']
    missing_cols = [col for col in required_cols if col not in team_tweets.columns]

    if missing_cols:
        print(f"Warning: Missing columns for feature engineering: {missing_cols}")
        # Try to fix or skip these features

    # Remove rows with empty cleaned text
    team_tweets = team_tweets[team_tweets['clean_text'].str.len() > 0].copy()

    # Create target variable: 1 for Patriots, 0 for Seahawks
    y = (team_tweets['team'] == 'patriots').astype(int)

    # Before vectorizing, ensure team_tweets isn't empty:
    if team_tweets.empty:
        print("No data left after filtering. Returning None.")
        return None, None, None, None

    # Check how many docs remain:
    print(f"Number of tweets: {len(team_tweets)}")

    # If you're debugging, print out some of the text:
    print("Sample cleaned text:\n", team_tweets['clean_text'].head(10).tolist())

    # Use TF-IDF vectorizer for text features
    print("Creating TF-IDF features...")
    # We limit to 2000 features to keep the model manageable
    vectorizer = TfidfVectorizer(
        max_features=2000,
        min_df=5,         # Minimum document frequency
        max_df=0.9,       # Maximum document frequency (ignore too common words)
        stop_words=stop_words  # Remove English stopwords
    )

    X_text = vectorizer.fit_transform(team_tweets['clean_text'])
    text_feature_names = vectorizer.get_feature_names_out()

    print(f"Created {X_text.shape[1]} text features")

    # Create metadata features
    print("Creating metadata features...")
    from scipy.sparse import hstack, csr_matrix

    # Start with text features
    feature_matrices = [X_text]
    feature_names = list(text_feature_names)

    # Time-related features (encoded as dummy variables)
    if 'time_period' in team_tweets.columns:
        time_features = pd.get_dummies(team_tweets['time_period'], prefix='time')
        feature_matrices.append(csr_matrix(time_features.values))
        feature_names.extend(time_features.columns.tolist())

    if 'quarter' in team_tweets.columns:
        quarter_features = pd.get_dummies(team_tweets['quarter'], prefix='quarter')
        feature_matrices.append(csr_matrix(quarter_features.values))
        feature_names.extend(quarter_features.columns.tolist())

    # Log-transform follower and retweet counts (to handle skewed distributions)
    if 'follower_count' in team_tweets.columns:
        log_followers = np.log1p(team_tweets['follower_count'].values).reshape(-1, 1)
        feature_matrices.append(csr_matrix(log_followers))
        feature_names.append('log_followers')

    if 'retweet_count' in team_tweets.columns:
        log_retweets = np.log1p(team_tweets['retweet_count'].values).reshape(-1, 1)
        feature_matrices.append(csr_matrix(log_retweets))
        feature_names.append('log_retweets')

    # Combine all features
    print("Combining features...")
    X = hstack(feature_matrices)

    print(f"Final feature matrix shape: {X.shape}")

    return X, y, feature_names, team_tweets.index

# Engineer features
X, y, feature_names, indices = engineer_features(team_tweets)

if y is None:
    print("\nNo data to display class distribution.")
else:
    print("\nClass distribution:")
    print(f"Patriots tweets: {sum(y == 1)}")
    print(f"Seahawks tweets: {sum(y == 0)}")
    ratio = sum(y == 1) / (sum(y == 0) or 1)
    print(f"Ratio (Patriots:Seahawks): {ratio:.2f}")



In [ ]:
"""
## Step 4: Model Training with Multiple Baselines
"""

def train_and_evaluate_models(X, y, feature_names):
    """
    Train and evaluate multiple models to establish baselines and find the best performer.

    Parameters:
    - X: Feature matrix
    - y: Target labels
    - feature_names: Names of the features

    Returns:
    - Dictionary containing trained models and their performance metrics
    """
    print("\n--- Model Training and Evaluation ---")

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"Training set: {X_train.shape[0]} samples")
    print(f"Testing set: {X_test.shape[0]} samples")

    # Handle class imbalance using random oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

    print(f"After oversampling - Training set: {X_train_resampled.shape[0]} samples")
    print(f"Class distribution after oversampling: {np.bincount(y_train_resampled)}")

    # Define models to train
    models = {
        'Random Baseline': DummyClassifier(strategy='uniform', random_state=42),
        'Most Frequent Baseline': DummyClassifier(strategy='most_frequent'),
        'Naive Bayes': MultinomialNB(),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42, C=1.0),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10, n_jobs=-1)
    }

    # Train and evaluate each model
    results = {}

    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train_resampled, y_train_resampled)

        # Evaluate on test set
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, target_names=['Seahawks', 'Patriots'])

        print(f"{name} Accuracy: {accuracy:.4f}")
        print(report)

        # Store results
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'predictions': y_pred,
            'report': report
        }

        # For models that support feature importance, extract and display
        if name == 'Logistic Regression':
            # Get coefficients and corresponding feature names
            coef = model.coef_[0]
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': coef
            })

            # Sort by absolute importance
            feature_importance['Abs_Importance'] = abs(feature_importance['Importance'])
            feature_importance = feature_importance.sort_values('Abs_Importance', ascending=False)

            # Display top features for each class
            print("\nTop features for Patriots (positive coefficients):")
            print(feature_importance[feature_importance['Importance'] > 0].head(10))

            print("\nTop features for Seahawks (negative coefficients):")
            print(feature_importance[feature_importance['Importance'] < 0].head(10))

            # Store feature importance
            results[name]['feature_importance'] = feature_importance

        elif name == 'Random Forest':
            # Get feature importance
            importances = model.feature_importances_
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values('Importance', ascending=False)

            print("\nTop Random Forest feature importances:")
            print(feature_importance.head(20))

            # Store feature importance
            results[name]['feature_importance'] = feature_importance

    return results

# Train and evaluate models
model_results = train_and_evaluate_models(X, y, feature_names)


In [ ]:
# STEP 5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytz
from datetime import datetime

###########################################
# STEP 5 (Time-Based Analysis) with Debug
###########################################

def analyze_time_patterns(team_tweets):
    """
    Analyze how fan behavior changes throughout the game.

    Parameters:
    - team_tweets: DataFrame containing preprocessed tweets

    Returns:
    - None (generates plots and prints analysis)
    """
    print("\n--- Time-Based Analysis ---")

    # Debug: check input size
    print(f"Initial team_tweets shape: {team_tweets.shape}")
    print("Columns:", list(team_tweets.columns))

    # Ensure 'datetime' is a proper datetime dtype
    if not pd.api.types.is_datetime64_any_dtype(team_tweets['datetime']):
        print("Converting 'datetime' column to datetime dtype...")
        team_tweets['datetime'] = pd.to_datetime(team_tweets['datetime'])
    else:
        print("'datetime' column is already a datetime dtype.")

    # Convert datetime to more manageable 5-minute intervals
    print("Creating 'time_bin' column (5-minute intervals)...")
    team_tweets['time_bin'] = team_tweets['datetime'].dt.floor('5min')
    print("time_bin head:\n", team_tweets['time_bin'].head())

    # Group by time bin and team, count tweets
    print("Grouping by (time_bin, team) to count tweets...")
    time_counts = team_tweets.groupby(['time_bin', 'team']).size().unstack(fill_value=0)

    # Debug: see a few rows of time_counts
    print("\ntime_counts head (showing first 10 rows):\n", time_counts.head(10))

    # Ensure columns for both teams exist
    if 'patriots' not in time_counts.columns:
        time_counts['patriots'] = 0
    if 'seahawks' not in time_counts.columns:
        time_counts['seahawks'] = 0

    # Plot tweet volume over time by team
    print("Plotting tweet volume over time by team...")
    plt.figure(figsize=(15, 7))
    time_counts['patriots'].plot(label='Patriots', color='blue')
    time_counts['seahawks'].plot(label='Seahawks', color='green')
    plt.title('Tweet Volume by Team Throughout Super Bowl Sunday')
    plt.xlabel('Time (PST)')
    plt.ylabel('Number of Tweets per 5 Minutes')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Add game period markers
    game_start = datetime(2015, 2, 1, 15, 30, tzinfo=pytz.timezone('America/Los_Angeles'))
    game_end   = datetime(2015, 2, 1, 19, 0, tzinfo=pytz.timezone('America/Los_Angeles'))

    plt.axvline(x=game_start, color='red', linestyle='--', alpha=0.7, label='Game Start')
    plt.axvline(x=game_end, color='red', linestyle='--', alpha=0.7, label='Game End')
    plt.savefig('tweet_volume_by_team.png')
    plt.show()
    print("Created tweet volume plot: tweet_volume_by_team.png")

    # Analyze engagement metrics over time
    print("\nAnalyzing engagement metrics...")
    team_tweets['engagement_ratio'] = team_tweets['retweet_count'] / (team_tweets['follower_count'] + 1)

    # Group by quarter and team for average engagement
    print("Grouping by (quarter, team) to compute mean engagement_ratio...")
    quarter_engagement = team_tweets.groupby(['quarter', 'team'])['engagement_ratio'].mean().unstack()

    print("\nquarter_engagement DataFrame:\n", quarter_engagement)

    # Plot engagement by quarter
    plt.figure(figsize=(12, 6))
    quarter_engagement.plot(kind='bar', figsize=(12, 6))
    plt.title('Average Engagement Ratio by Quarter and Team')
    plt.xlabel('Game Quarter')
    plt.ylabel('Average Engagement Ratio (Retweets / Followers)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('engagement_by_quarter.png')
    plt.show()
    print("Created engagement analysis plot: engagement_by_quarter.png")

    # Compute ratio of patriots to seahawks tweets
    print("\nComputing ratio of patriots to seahawks tweets per 5-min bin...")
    time_counts['ratio'] = time_counts['patriots'] / (time_counts['seahawks'] + 1)

    # Debug: see first 10 ratio values
    print("\nFirst 10 ratio values:\n", time_counts['ratio'].head(10))

    # Option A: Filter out bins with very few total tweets
    time_counts['total'] = time_counts['patriots'] + time_counts['seahawks']
    filtered = time_counts[time_counts['total'] >= 5].copy()
    filtered['ratio'] = filtered['patriots'] / (filtered['seahawks'] + 1)

    # Identify key moments with ratio > 1.5 or ratio < 0.67
    key_moments = filtered[
        (filtered['ratio'] > 1.5) | (filtered['ratio'] < 0.67)
    ].index.tolist()
    key_moments.sort()

    print("\n--- Approach 1: Filtered bins (total >= 5) ---")
    print(f"Found {len(key_moments)} key moments with significant difference (ratio>1.5 or ratio<0.67)")

    if key_moments:
        print("Showing up to 10 such moments:")
        for moment in key_moments[:10]:
            ratio_val = filtered.loc[moment, 'ratio']
            # Decide who is more active
            if ratio_val == 0:
                more_active = "Seahawks (no Patriots tweets)"
            elif np.isinf(ratio_val):
                more_active = "Patriots (no Seahawks tweets)"
            else:
                more_active = "Patriots" if ratio_val > 1 else "Seahawks"
            factor = max(ratio_val, 1/ratio_val) if ratio_val != 0 else np.inf
            print(f"{moment} - ratio={ratio_val:.3f} => {more_active} were {factor:.2f}x more active")

    # Option B: Keep all bins, but handle infinite or zero ratios explicitly
    all_time_counts = time_counts.copy()
    all_time_counts['ratio'] = all_time_counts['patriots'] / (all_time_counts['seahawks'] + 1)
    all_key_moments = all_time_counts[
        (all_time_counts['ratio'] > 1.5) | (all_time_counts['ratio'] < 0.67)
    ].index.tolist()
    all_key_moments.sort()

    print("\n--- Approach 2: All bins (including total < 5) ---")
    print(f"Found {len(all_key_moments)} key moments with significant difference in ratio")
    if all_key_moments:
        print("Showing up to 10 such moments:")
        for moment in all_key_moments[:10]:
            ratio_val = all_time_counts.loc[moment, 'ratio']
            if ratio_val == 0:
                # Means patriots=0, seahawks>0
                print(f"{moment}: Seahawks fans were MUCH more active (no Patriots tweets).")
            elif np.isinf(ratio_val):
                # Means seahawks=0, patriots>0
                print(f"{moment}: Patriots fans were MUCH more active (no Seahawks tweets).")
            else:
                more_active = "Patriots" if ratio_val > 1 else "Seahawks"
                factor = max(ratio_val, 1/ratio_val)
                print(f"{moment}: {more_active} fans were {factor:.2f}x more active")

# Example usage (assuming you have a DataFrame called team_tweets)
analyze_time_patterns(team_tweets)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

###########################################
# Step 6: Keyword Analysis by Game Period
###########################################

def analyze_keywords_by_period(team_tweets):
    """
    Analyze how keywords used by fans change throughout the game.

    Parameters:
    - team_tweets: DataFrame containing preprocessed tweets

    Returns:
    - None (prints analysis and generates plots)
    """
    print("\n--- Keyword Analysis by Game Period ---")

    # Define periods of interest
    periods = ['pre_game', 'during_game', 'post_game']
    period_keywords = {}

    for period in periods:
        print(f"\nProcessing period: {period}")
        # Filter tweets by period
        period_data = team_tweets[team_tweets['time_period'] == period]
        print(f"Total tweets in {period}: {len(period_data)}")

        # If not enough data, skip this period
        if len(period_data) < 10:
            print(f"Skipping {period} period due to insufficient tweets (<10).")
            continue

        # Separate tweets by team
        patriots_tweets = period_data[period_data['team'] == 'patriots']['clean_text']
        seahawks_tweets = period_data[period_data['team'] == 'seahawks']['clean_text']

        print(f"Patriots tweets count in {period}: {len(patriots_tweets)}")
        print(f"Seahawks tweets count in {period}: {len(seahawks_tweets)}")

        # Create CountVectorizer with lower min_df (e.g., 2) for small datasets
        vectorizer_params = {
            'max_features': 100,
            'min_df': 2,
            'stop_words': stop_words  # ensure stop_words is defined in your environment
        }

        # Process Patriots tweets
        patriots_term_counts = []
        if len(patriots_tweets) > 10:
            print("\nAnalyzing keywords for Patriots fans...")
            try:
                vectorizer_pats = CountVectorizer(**vectorizer_params)
                patriots_dtm = vectorizer_pats.fit_transform(patriots_tweets)
                patriots_terms = vectorizer_pats.get_feature_names_out()
                patriots_counts = patriots_dtm.sum(axis=0).A1
                patriots_term_counts = sorted(zip(patriots_terms, patriots_counts), key=lambda x: x[1], reverse=True)
                print(f"Top terms for Patriots fans ({period}):")
                for term, count in patriots_term_counts[:15]:
                    print(f"- {term}: {count}")
            except ValueError as e:
                print(f"Error processing Patriots tweets in {period}: {e}")
        else:
            print(f"Not enough Patriots tweets in {period} to analyze.")

        # Process Seahawks tweets
        seahawks_term_counts = []
        if len(seahawks_tweets) > 10:
            print("\nAnalyzing keywords for Seahawks fans...")
            try:
                vectorizer_hawks = CountVectorizer(**vectorizer_params)
                seahawks_dtm = vectorizer_hawks.fit_transform(seahawks_tweets)
                seahawks_terms = vectorizer_hawks.get_feature_names_out()
                seahawks_counts = seahawks_dtm.sum(axis=0).A1
                seahawks_term_counts = sorted(zip(seahawks_terms, seahawks_counts), key=lambda x: x[1], reverse=True)
                print(f"Top terms for Seahawks fans ({period}):")
                for term, count in seahawks_term_counts[:15]:
                    print(f"- {term}: {count}")
            except ValueError as e:
                print(f"Error processing Seahawks tweets in {period}: {e}")
        else:
            print(f"Not enough Seahawks tweets in {period} to analyze.")

        period_keywords[period] = {
            'patriots': patriots_term_counts[:20] if len(patriots_tweets) > 10 else [],
            'seahawks': seahawks_term_counts[:20] if len(seahawks_tweets) > 10 else []
        }

    # Create visualizations for keyword differences for each period with sufficient data
    for period in period_keywords:
        if not period_keywords[period]['patriots'] or not period_keywords[period]['seahawks']:
            print(f"Skipping visualization for {period} due to insufficient keyword data.")
            continue

        plt.figure(figsize=(12, 6))

        # Patriots keywords bar chart
        plt.subplot(1, 2, 1)
        patriots_words = [word for word, count in period_keywords[period]['patriots']][:10]
        patriots_counts = [count for word, count in period_keywords[period]['patriots']][:10]
        y_pos = np.arange(len(patriots_words))
        plt.barh(y_pos, patriots_counts, color='blue')
        plt.yticks(y_pos, patriots_words)
        plt.title(f'Patriots Fan Keywords ({period})')
        plt.xlabel('Count')

        # Seahawks keywords bar chart
        plt.subplot(1, 2, 2)
        seahawks_words = [word for word, count in period_keywords[period]['seahawks']][:10]
        seahawks_counts = [count for word, count in period_keywords[period]['seahawks']][:10]
        y_pos = np.arange(len(seahawks_words))
        plt.barh(y_pos, seahawks_counts, color='green')
        plt.yticks(y_pos, seahawks_words)
        plt.title(f'Seahawks Fan Keywords ({period})')
        plt.xlabel('Count')

        plt.tight_layout()
        filename = f'keywords_{period}.png'
        plt.savefig(filename)
        plt.show()
        print(f"Created keyword visualization: {filename}")

    return period_keywords

# Run the keyword analysis function
keyword_results = analyze_keywords_by_period(team_tweets)


In [ ]:
"""
## Step 7: Create Game Timeline Visualization
"""

def create_game_timeline_visualization(team_tweets):
    """
    Create a timeline visualization showing key moments in the game and corresponding tweet activity.

    Parameters:
    - team_tweets: DataFrame containing preprocessed tweets

    Returns:
    - None (generates plot)
    """
    print("\n" + "="*80)
    print("                 GAME TIMELINE VISUALIZATION                        ")
    print("="*80)

    # Define key game events for Super Bowl 49
    key_events = [
        {'time': datetime(2015, 2, 1, 15, 30, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Kickoff', 'team': 'neutral'},
        {'time': datetime(2015, 2, 1, 15, 45, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Patriots TD - Brady to LaFell', 'team': 'patriots'},
        {'time': datetime(2015, 2, 1, 15, 55, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Seahawks TD - Lynch 3-yard run', 'team': 'seahawks'},
        {'time': datetime(2015, 2, 1, 16, 8, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Patriots TD - Brady to Gronkowski', 'team': 'patriots'},
        {'time': datetime(2015, 2, 1, 16, 15, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Seahawks TD - Wilson to Matthews', 'team': 'seahawks'},
        {'time': datetime(2015, 2, 1, 16, 35, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Halftime Show', 'team': 'neutral'},
        {'time': datetime(2015, 2, 1, 17, 10, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Seahawks TD - Baldwin from Wilson', 'team': 'seahawks'},
        {'time': datetime(2015, 2, 1, 18, 0, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Patriots TD - Brady to Amendola', 'team': 'patriots'},
        {'time': datetime(2015, 2, 1, 18, 10, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Patriots TD - Brady to Edelman', 'team': 'patriots'},
        {'time': datetime(2015, 2, 1, 18, 55, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Butler Interception', 'team': 'patriots'},
        {'time': datetime(2015, 2, 1, 19, 0, tzinfo=pytz.timezone('America/Los_Angeles')),
         'event': 'Game End - Patriots Win', 'team': 'patriots'}
    ]

    # Create a DataFrame for events
    events_df = pd.DataFrame(key_events)

    # Group tweets by 2-minute bins and team
    team_tweets['time_bin'] = team_tweets['datetime'].dt.floor('2min')
    time_counts = team_tweets.groupby(['time_bin', 'team']).size().unstack(fill_value=0)

    # Ensure both teams are in columns
    if 'patriots' not in time_counts.columns:
        time_counts['patriots'] = 0
    if 'seahawks' not in time_counts.columns:
        time_counts['seahawks'] = 0

    # Calculate total tweet volume
    time_counts['total'] = time_counts['patriots'] + time_counts['seahawks']

    # Plot the game timeline
    plt.figure(figsize=(20, 10))

    # Plot tweet volume
    plt.plot(time_counts.index, time_counts['patriots'], label='Patriots Tweets', color='blue', alpha=0.7)
    plt.plot(time_counts.index, time_counts['seahawks'], label='Seahawks Tweets', color='green', alpha=0.7)

    # Add event markers
    for _, event in events_df.iterrows():
        color = 'blue' if event['team'] == 'patriots' else 'green' if event['team'] == 'seahawks' else 'gray'
        plt.axvline(x=event['time'], color=color, linestyle='--', alpha=0.5)

        # Add text label for event
        # Calculate y position based on which team the event is related to
        if event['team'] == 'patriots':
            y_pos = time_counts['total'].max() * 0.8
        elif event['team'] == 'seahawks':
            y_pos = time_counts['total'].max() * 0.7
        else:
            y_pos = time_counts['total'].max() * 0.9

        plt.text(event['time'], y_pos, event['event'], rotation=45, fontsize=9,
                 ha='right', color=color, bbox=dict(facecolor='white', alpha=0.8))

    # Customize plot
    plt.title('Super Bowl 49 Timeline: Tweet Volume and Key Events', fontsize=16)
    plt.xlabel('Time (PST)', fontsize=12)
    plt.ylabel('Number of Tweets per 2-Minute Window', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)

    # Add game period markers
    game_periods = [
        (datetime(2015, 2, 1, 15, 30, tzinfo=pytz.timezone('America/Los_Angeles')), 'Q1 Start'),
        (datetime(2015, 2, 1, 16, 15, tzinfo=pytz.timezone('America/Los_Angeles')), 'Q2 Start'),
        (datetime(2015, 2, 1, 17, 0, tzinfo=pytz.timezone('America/Los_Angeles')), 'Q3 Start'),
        (datetime(2015, 2, 1, 17, 45, tzinfo=pytz.timezone('America/Los_Angeles')), 'Q4 Start'),
        (datetime(2015, 2, 1, 19, 0, tzinfo=pytz.timezone('America/Los_Angeles')), 'Game End')
    ]

    for time, label in game_periods:
        plt.axvline(x=time, color='black', linestyle='-', alpha=0.3)
        plt.text(time, time_counts['total'].max() * 0.95, label, fontsize=10,
                 ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))

    # Add annotation for the famous interception
    interception_time = datetime(2015, 2, 1, 18, 55, tzinfo=pytz.timezone('America/Los_Angeles'))

    # Find the nearest index to the interception time
    nearest_times = time_counts.index[time_counts.index.get_indexer([interception_time], method='nearest')]
    if len(nearest_times) > 0:
        nearest_time = nearest_times[0]
        if nearest_time in time_counts.index:
            plt.annotate('Game-winning interception by Butler',
                        xy=(interception_time, time_counts.loc[nearest_time, 'total']),
                        xytext=(interception_time, time_counts['total'].max() * 0.6),
                        arrowprops=dict(facecolor='black', shrink=0.05, width=1.5, headwidth=8),
                        fontsize=12, ha='center', bbox=dict(facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.savefig('superbowl_game_timeline.png', dpi=300)
    plt.show()
    print("Created game timeline visualization: superbowl_game_timeline.png")

# Create game timeline visualization
create_game_timeline_visualization(team_tweets)


In [ ]:
"""
## Step 8: Generate Final Report
"""

def generate_final_report(team_tweets, model_results):
    """
    Generate a final summary of findings.

    Parameters:
    - team_tweets: DataFrame containing preprocessed tweets
    - model_results: Dictionary containing model results

    Returns:
    - None (prints final report)
    """
    print("\n" + "="*80)
    print("                        FINAL PROJECT REPORT                        ")
    print("="*80)

    print("\n1. TASK DESCRIPTION")
    print("-"*80)
    print("This project aimed to classify tweets from Super Bowl 49 as supporting either")
    print("the New England Patriots or Seattle Seahawks based on tweet content and metadata.")
    print("The project also analyzed how fan language and engagement patterns changed throughout the game.")

    print("\n2. DATASET OVERVIEW")
    print("-"*80)
    patriots_count = (team_tweets['team'] == 'patriots').sum()
    seahawks_count = (team_tweets['team'] == 'seahawks').sum()
    total_tweets = len(team_tweets)

    print(f"Total tweets analyzed: {total_tweets}")
    print(f"Patriots tweets: {patriots_count} ({patriots_count/total_tweets*100:.1f}%)")
    print(f"Seahawks tweets: {seahawks_count} ({seahawks_count/total_tweets*100:.1f}%)")

    print("\nTime distribution:")
    print(team_tweets['time_period'].value_counts())

    print("\n3. FEATURE ENGINEERING")
    print("-"*80)
    print("Features used in this analysis:")
    print("- Text features: TF-IDF vectors from cleaned tweet text")
    print("- Temporal features: Game period (pre-game, during game, post-game)")
    print("- Game progress features: Quarter information")
    print("- Engagement features: Follower count and retweet count")

    print("\n4. MODEL PERFORMANCE")
    print("-"*80)
    # Get best model based on accuracy
    best_model_name = max(model_results.items(), key=lambda x: x[1]['accuracy'])[0]
    best_model_accuracy = model_results[best_model_name]['accuracy']

    print(f"Best performing model: {best_model_name}")
    print(f"Accuracy: {best_model_accuracy:.4f}")
    print("\nModel comparison:")

    for name, results in model_results.items():
        print(f"- {name}: {results['accuracy']:.4f}")

    print("\n5. KEY FINDINGS")
    print("-"*80)

    # Find most important features for best model
    if 'feature_importance' in model_results[best_model_name]:
        fi = model_results[best_model_name]['feature_importance']
        if best_model_name == 'Logistic Regression':
            # For logistic regression, positive = Patriots, negative = Seahawks
            print("Most predictive words for Patriots fans:")
            patriots_words = fi[fi['Importance'] > 0].head(10)['Feature'].tolist()
            print(", ".join(patriots_words))

            print("\nMost predictive words for Seahawks fans:")
            seahawks_words = fi[fi['Importance'] < 0].head(10)['Feature'].tolist()
            print(", ".join(seahawks_words))

        elif best_model_name == 'Random Forest':
            # For random forest, we just have overall feature importance
            print("Most important features for team classification:")
            top_features = fi.head(15)['Feature'].tolist()
            print(", ".join(top_features))

    # Time-based observations
    print("\nObservations about fan behavior over time:")
    print("- Fan engagement varied significantly throughout the game")
    print("- The volume of tweets increased dramatically during key game moments")
    print("- Post-game tweets showed distinct language patterns compared to pre-game tweets")

    print("\n6. CONCLUSIONS")
    print("-"*80)
    print("This analysis demonstrates that fan language on Twitter during a major sporting event")
    print("has distinct patterns that can be used to identify team allegiance. The model was able")
    print("to distinguish between Patriots and Seahawks fans with good accuracy beyond just hashtag usage.")
    print("\nFan behavior changed throughout the game, with engagement patterns reflecting the")
    print("dramatic nature of Super Bowl 49, which ended with a game-changing interception.")
    print("\nThis type of analysis could be valuable for:")
    print("- Understanding fan communities and their language")
    print("- Tracking real-time reactions to sporting events")
    print("- Identifying influential content during high-engagement periods")

# Generate the final report
generate_final_report(team_tweets, model_results)

print("\n" + "="*80)
print("                        PROJECT COMPLETE                              ")
print("="*80)
print("This project demonstrates how to classify tweets by team affiliation and")
print("analyze fan behavior patterns during Super Bowl 49. The approach is straightforward")
print("yet effective, providing meaningful insights without complex deep learning models.")
print("\nKey files generated:")
print("- tweet_volume_by_team.png: Shows tweet volume over time by team")
print("- engagement_by_quarter.png: Shows engagement metrics by quarter")
print("- keywords_*.png: Shows popular keywords by game period")
print("- superbowl_game_timeline.png: Shows tweet activity aligned with game events")
print("\nRecommendations for further analysis:")
print("1. Incorporate sentiment analysis to track emotional changes during key moments")
print("2. Analyze network effects by looking at retweet patterns")
print("3. Compare language differences between casual and dedicated fans")
print("4. Extend analysis to include postgame media coverage correlation")

In [ ]:
# Step 8.1: Training SVM (LinearSVC) with Train/Test Split and Oversampling

# Import necessary libraries for splitting, oversampling, and evaluation
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# Ensure that X and y have been defined from your feature engineering step.
# For example, if you have already run: X, y, feature_names, indices = engineer_features(team_tweets)
print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])

# Handle class imbalance using RandomOverSampler on the training set
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
print("Oversampled training samples:", X_train_resampled.shape[0])
print("Class distribution after oversampling:", np.bincount(y_train_resampled))

# Train the SVM model using LinearSVC
print("\nStep 8.1: Training SVM (LinearSVC)...")
svm_model = LinearSVC(random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_svm = svm_model.predict(X_test)

# Evaluate performance: accuracy and classification report
svm_accuracy = accuracy_score(y_test, y_pred_svm)
svm_report = classification_report(y_test, y_pred_svm, target_names=['Seahawks', 'Patriots'])

print("SVM Accuracy: {:.4f}".format(svm_accuracy))
print(svm_report)

# Optionally, integrate SVM results into an existing model_results dictionary if desired:
try:
    model_results['SVM'] = {
        'model': svm_model,
        'accuracy': svm_accuracy,
        'predictions': y_pred_svm,
        'report': svm_report
    }
except NameError:
    # model_results might not be defined if this is a standalone block
    model_results = {'SVM': {
        'model': svm_model,
        'accuracy': svm_accuracy,
        'predictions': y_pred_svm,
        'report': svm_report
    }}


In [ ]:
# Step 8.2: Training MLPClassifier (Optimized for Speed)

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.2: Training MLPClassifier (Optimized)...")

# Option 1: If you have enough memory, convert sparse matrices to dense arrays
# Uncomment the next two lines if memory is sufficient.
# X_train_resampled_dense = X_train_resampled.toarray()
# X_test_dense = X_test.toarray()

# Option 2: If you prefer to work with the sparse matrices directly, be aware that MLPClassifier will convert them internally,
# which may slow down training.

# Initialize the MLP classifier with reduced iterations and early stopping enabled
mlp_model = MLPClassifier(random_state=42,
                          max_iter=100,         # Reduced number of iterations
                          early_stopping=True,  # Enable early stopping based on validation score
                          verbose=True)         # Verbose output to monitor progress

# Use the dense arrays if you converted; otherwise, use the original sparse matrices.
# For example, if using dense arrays:
# mlp_model.fit(X_train_resampled_dense, y_train_resampled)
# y_pred_mlp = mlp_model.predict(X_test_dense)

# Otherwise, fit directly (MLPClassifier will convert internally)
mlp_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_mlp = mlp_model.predict(X_test)

# Evaluate performance: accuracy and classification report
mlp_accuracy = accuracy_score(y_test, y_pred_mlp)
mlp_report = classification_report(y_test, y_pred_mlp, target_names=['Seahawks', 'Patriots'])

print("MLP Accuracy: {:.4f}".format(mlp_accuracy))
print(mlp_report)

# Optionally, add the MLP results to your model_results dictionary for comparison:
try:
    model_results['MLP'] = {
        'model': mlp_model,
        'accuracy': mlp_accuracy,
        'predictions': y_pred_mlp,
        'report': mlp_report
    }
except NameError:
    model_results = {'MLP': {
        'model': mlp_model,
        'accuracy': mlp_accuracy,
        'predictions': y_pred_mlp,
        'report': mlp_report
    }}


In [ ]:
# Step 8.3: Training XGBoost Classifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.3: Training XGBoost Classifier...")

# Initialize the XGBoost classifier.
# use_label_encoder=False suppresses a warning; eval_metric='logloss' is set for binary classification.
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model on the oversampled training set
xgb_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate performance: accuracy and classification report
xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
xgb_report = classification_report(y_test, y_pred_xgb, target_names=['Seahawks', 'Patriots'])

print("XGBoost Accuracy: {:.4f}".format(xgb_accuracy))
print(xgb_report)

# Optionally, add the XGBoost results to your model_results dictionary:
try:
    model_results['XGBoost'] = {
        'model': xgb_model,
        'accuracy': xgb_accuracy,
        'predictions': y_pred_xgb,
        'report': xgb_report
    }
except NameError:
    model_results = {'XGBoost': {
        'model': xgb_model,
        'accuracy': xgb_accuracy,
        'predictions': y_pred_xgb,
        'report': xgb_report
    }}


In [ ]:
# Step 8.4: Training Voting Classifier Ensemble
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.4: Training Voting Classifier Ensemble...")

# Define the base estimators as a list of (name, estimator) tuples
estimators = [
    ('svm', LinearSVC(random_state=42)),
    ('xgb', xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
    ('mlp', MLPClassifier(random_state=42, max_iter=100, early_stopping=True, verbose=False))
]

# Create a VotingClassifier ensemble using hard voting (majority rule)
voting_clf = VotingClassifier(estimators=estimators, voting='hard')

# Fit the ensemble on the oversampled training data
voting_clf.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_voting = voting_clf.predict(X_test)

# Evaluate performance: accuracy and classification report
voting_accuracy = accuracy_score(y_test, y_pred_voting)
voting_report = classification_report(y_test, y_pred_voting, target_names=['Seahawks', 'Patriots'])

print("Voting Classifier Accuracy: {:.4f}".format(voting_accuracy))
print(voting_report)

# Optionally, integrate the ensemble results into your model_results dictionary:
try:
    model_results['Voting'] = {
        'model': voting_clf,
        'accuracy': voting_accuracy,
        'predictions': y_pred_voting,
        'report': voting_report
    }
except NameError:
    model_results = {'Voting': {
        'model': voting_clf,
        'accuracy': voting_accuracy,
        'predictions': y_pred_voting,
        'report': voting_report
    }}


In [ ]:
# Step 8.5.1: Training AdaBoost Classifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.5.1: Training AdaBoost Classifier...")

ada_model = AdaBoostClassifier(random_state=42, n_estimators=100)
ada_model.fit(X_train_resampled, y_train_resampled)
y_pred_ada = ada_model.predict(X_test)
ada_accuracy = accuracy_score(y_test, y_pred_ada)
print("AdaBoost Accuracy: {:.4f}".format(ada_accuracy))
print(classification_report(y_test, y_pred_ada, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.5.2: Training LightGBM Classifier

import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.5.2: Training LightGBM Classifier...")

lgb_model = lgb.LGBMClassifier(random_state=42, n_estimators=100)
lgb_model.fit(X_train_resampled, y_train_resampled)
y_pred_lgb = lgb_model.predict(X_test)
lgb_accuracy = accuracy_score(y_test, y_pred_lgb)
print("LightGBM Accuracy: {:.4f}".format(lgb_accuracy))
print(classification_report(y_test, y_pred_lgb, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.5.3: Training CatBoost Classifier
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.5.3: Training CatBoost Classifier...")

# Suppress verbose output with verbose=0
cat_model = CatBoostClassifier(random_seed=42, iterations=100, verbose=0)
cat_model.fit(X_train_resampled, y_train_resampled)
y_pred_cat = cat_model.predict(X_test)
cat_accuracy = accuracy_score(y_test, y_pred_cat)
print("CatBoost Accuracy: {:.4f}".format(cat_accuracy))
print(classification_report(y_test, y_pred_cat, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.5.4: Training ExtraTrees Classifier

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.5.4: Training ExtraTrees Classifier...")

et_model = ExtraTreesClassifier(random_state=42, n_estimators=100)
et_model.fit(X_train_resampled, y_train_resampled)
y_pred_et = et_model.predict(X_test)
et_accuracy = accuracy_score(y_test, y_pred_et)
print("ExtraTrees Accuracy: {:.4f}".format(et_accuracy))
print(classification_report(y_test, y_pred_et, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.5.5: Training K-Nearest Neighbors (KNN) Classifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.5.5: Training K-Nearest Neighbors (KNN) Classifier...")

# KNN does not support sparse matrices well, so convert to dense arrays.
X_train_dense = X_train_resampled.toarray()
X_test_dense = X_test.toarray()

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_dense, y_train_resampled)
y_pred_knn = knn_model.predict(X_test_dense)
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print("KNN Accuracy: {:.4f}".format(knn_accuracy))
print(classification_report(y_test, y_pred_knn, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.6: Training Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.6: Training Decision Tree Classifier...")

dt_model = DecisionTreeClassifier(random_state=42, max_depth=10)
dt_model.fit(X_train_resampled, y_train_resampled)
y_pred_dt = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy: {:.4f}".format(dt_accuracy))
print(classification_report(y_test, y_pred_dt, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.7: Training Ridge Classifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.7: Training Ridge Classifier...")

ridge_model = RidgeClassifier(random_state=42)
ridge_model.fit(X_train_resampled, y_train_resampled)
y_pred_ridge = ridge_model.predict(X_test)
ridge_accuracy = accuracy_score(y_test, y_pred_ridge)
print("Ridge Classifier Accuracy: {:.4f}".format(ridge_accuracy))
print(classification_report(y_test, y_pred_ridge, target_names=['Seahawks', 'Patriots']))


In [ ]:
# Step 8.8: Training Stacking Classifier Ensemble
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

print("\nStep 8.8: Training Stacking Classifier Ensemble...")

# Define base estimators as a list of (name, estimator) tuples.
base_estimators = [
    ('svm', LinearSVC(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42, max_depth=10)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# Use Logistic Regression as the final (meta) estimator.
stack_model = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression(random_state=42))

# Fit the stacking classifier on the oversampled training data.
stack_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set.
y_pred_stack = stack_model.predict(X_test)

stack_accuracy = accuracy_score(y_test, y_pred_stack)
print("Stacking Classifier Accuracy: {:.4f}".format(stack_accuracy))
print(classification_report(y_test, y_pred_stack, target_names=['Seahawks', 'Patriots']))


In [ ]:
## Step 8: Generate Final Report

def generate_final_report(team_tweets, model_results):
    """
    Generate a final summary report of the project.

    Parameters:
    - team_tweets: DataFrame containing the preprocessed tweets.
    - model_results: Dictionary containing evaluation results from all models.

    Returns:
    - None. Prints the final report.
    """
    print("\n" + "="*80)
    print("                        FINAL PROJECT REPORT                        ")
    print("="*80)

    # 1. Task Description
    print("\n1. TASK DESCRIPTION")
    print("-"*80)
    print("This project aimed to classify tweets from Super Bowl 49 as supporting either")
    print("the New England Patriots or the Seattle Seahawks using both tweet content and metadata.")
    print("It also analyzed how fan language and engagement patterns changed throughout the game.")

    # 2. Dataset Overview
    print("\n2. DATASET OVERVIEW")
    print("-"*80)
    total_tweets = len(team_tweets)
    patriots_count = (team_tweets['team'] == 'patriots').sum()
    seahawks_count = (team_tweets['team'] == 'seahawks').sum()
    print(f"Total tweets analyzed: {total_tweets}")
    print(f"Patriots tweets: {patriots_count} ({patriots_count/total_tweets*100:.1f}%)")
    print(f"Seahawks tweets: {seahawks_count} ({seahawks_count/total_tweets*100:.1f}%)")

    print("\nTime distribution of tweets:")
    print(team_tweets['time_period'].value_counts())

    # 3. Feature Engineering Summary
    print("\n3. FEATURE ENGINEERING")
    print("-"*80)
    print("The analysis utilized the following engineered features:")
    print(" - TF-IDF vectors from cleaned tweet text")
    print(" - Temporal features indicating game periods (pre-game, during game, post-game)")
    print(" - Game progress information (quarter of play)")
    print(" - Engagement metrics such as follower count and retweet count")

    # 4. Model Performance Summary
    print("\n4. MODEL PERFORMANCE")
    print("-"*80)
    # Identify best performing model
    best_model_name = max(model_results.items(), key=lambda x: x[1]['accuracy'])[0]
    best_model_accuracy = model_results[best_model_name]['accuracy']
    print(f"Best performing model: {best_model_name}")
    print(f"Accuracy: {best_model_accuracy:.4f}\n")

    print("Individual Model Performances:")
    for name, results in model_results.items():
        print(f" - {name}: Accuracy = {results['accuracy']:.4f}")

    # 5. Key Findings and Observations
    print("\n5. KEY FINDINGS")
    print("-"*80)
    print(" - Fan language and engagement on Twitter varied significantly over the course of the game.")
    print(" - Tweet volume spiked during critical game moments, reflecting real-time fan reactions.")
    print(" - Models leveraging both text and metadata achieved accuracies in the mid-70%s,")
    print("   and ensemble methods further improved performance and robustness.")

    # 6. Conclusions and Future Directions
    print("\n6. CONCLUSIONS AND FUTURE WORK")
    print("-"*80)
    print("This project demonstrates that social media data can effectively indicate team allegiance and")
    print("capture dynamic fan behavior during a major sporting event. With models such as SVM, MLP,")
    print("XGBoost, AdaBoost, LightGBM, CatBoost, ExtraTrees, and ensemble methods,")
    print("we achieved competitive accuracies (approximately 63% to 76%) across different approaches.")
    print("\nFuture enhancements could include:")
    print("1. Incorporating sentiment analysis to track emotional changes during key game moments.")
    print("2. Analyzing retweet and follower networks to identify influential fan accounts.")
    print("3. Comparing language differences between casual and dedicated fans.")
    print("4. Extending the analysis to include postgame media coverage and broader fan interactions.")

    print("\n" + "="*80)
    print("                        PROJECT COMPLETE                              ")
    print("="*80)
    print("Key files generated:")
    print(" - tweet_volume_by_team.png: Visualizes tweet volume over time by team")
    print(" - engagement_by_quarter.png: Shows engagement metrics by game quarter")
    print(" - keywords_*.png: Highlights popular keywords by game period")
    print(" - superbowl_game_timeline.png: Aligns tweet activity with key game events")
    print("\nRecommendations for further analysis:")
    print("1. Extend sentiment analysis to monitor emotional responses.")
    print("2. Study network effects via retweet and follower analysis.")
    print("3. Explore deeper neural network architectures for improved feature extraction.")
    print("4. Correlate social media trends with traditional media coverage.")

# Generate the final report
generate_final_report(team_tweets, model_results)
